<a href="https://colab.research.google.com/github/amlannag6/fnfta_DataProject/blob/main/FNFTA_DATAMINING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Packages

In [ ]:
pip install pytesseract

In [ ]:
pip install PdfReader

In [ ]:
pip install PyPDF2

# FNFTA DATA EXTRACTION - version 20.1

1.   NTLK (Natural Language Toolkit Model) -AI for parameter predection
2.  Google Vison



In [ ]:
pip install google-cloud-vision

In [ ]:
import requests
from PIL import Image
from io import BytesIO
from PyPDF2 import PdfReader
from nltk.corpus import wordnet
import pytesseract
from google.cloud import vision


# Function to extract text from an image using Google Cloud Vision OCR
def extract_text_from_image(image_path):
    client = vision.ImageAnnotatorClient()

    with open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations

    extracted_text = ""
    for text in texts:
        extracted_text += text.description + '\n'

    return extracted_text


# Function to extract text from an image using pytesseract OCR
def extract_text_from_image_pytesseract(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text


# Function to extract text from a readable PDF
def extract_text_from_pdf(pdf_path):
    pdf_file = open(pdf_path, 'rb')
    pdf_reader = PdfReader(pdf_file)

    num_pages = len(pdf_reader.pages)

    text_data = []
    for page_num in range(num_pages):
        page = pdf_reader.pages[page_num]
        text_data.append(page.extract_text())

    pdf_file.close()
    return '\n'.join(text_data)


def extract_financial_data(text):
    data = {
       'Restricted cash - capital projects (Note 3)': None,
        'Cash and Cash equivalents': None,
        'Cash (Note 2)': None,
        'Other revenue': None,
        'Economic Development': None,
        'Housing': None,
        'Cash and Equivalents': None,
        'Restricted Cash': None,
        'Accounts Receivables (AR)': None,
        'Inventory': None,
        'Long-term Investments': None,
        'Investment in GBEs': None,
        'Related Party Loans': None,
        'Investment in Trust Funds': None,
        'Other Assets': None,
        'Total Financial Assets': None,
        'Bank Indebtedness': None,
        'Accounts Payable': None,
        'Related Part Loans': None,
        'Deferred Revenue': None,
        'Short-term Debt': None,
        'Long-term Debt': None,
        'Other Liabilities': None,
        'Total Financial Liabilities': None,
        'Net Financial Assets': None,
        'Total Net Financial Assets': None,
        'Non-Financial Assets': None,
        'Tangible Capital Assets': None,
        'Prepaid Expenses': None,
        'Other non-Financial Assets': None,
        'Total non-Financial Assets': None,
        'Accumulated Surplus': None,
        'Income Statement': None,
        'Revenue': None,
        'Indigenous and Norther Affairs Canada (ISC)': None,
        'First Nations Health Authority': None,
        'Canadian Mortgage and Housing Corporation (CMHC)': None,
        'Health Canada': None,
        'Other Federal Transfer Revenue': None,
        'Provincial Transfer Revenue': None,
        'Tribal Government Transfer Revenue': None,
        'Other First Nation Entity Transfer Revenue': None,
        'Other Grants': None,
        'Royalties': None,
        'Rent/Lease Revenue': None,
        'Interest': None,
        'Property Tax': None,
        'Sales Tax': None,
        'Net GBE Income': None,
        'Gross Business Sales (non-GBE)': None,
        'User Fees': None,
        'Trust Fund': None,
        'Other Revenue': None,
        'Total Revenue': None,
        'Expenses': None,
        'Administration General': None,
        'Education': None,
        'Social Services and Development': None,
        'Capital and Maintenance': None,
        'Community Services': None,
        'Housing': None,
        'Land Resource Management': None,
        'Economic Development': None,
        'Business Expenses (Businesses directly controlled by the Nation)': None,
        'Health': None,
        'Combine Departments': None,
        'Other Expenses': None,
        'Total Expenses': None,
        'Surplus/Deficit': None,
        'Surplus in year': None,
        'Cash Flows': None,
        'Net Operating Cash Flows': None,
        'Gross Investment Cash Inflows': None,
        'Gross Investment Cash Outflows': None,
        'Net Investment Cash Flows': None,
        'Gross capital Cash inflows': None,
        'Net capital cash flows': None,
        'Gross financial cash inflows': None,
        'Gross financial cash outflows': None,
        'Net financial cash flows': None,
        'Total change in cash during year': None,
        'Cash balance beginning of year': None,
        'Cash balance end of year': None,
        'GBE Financial Information': None,
        'GBE Assets': None,
        'GBE Liabilities': None,
        'GBE Equity': None,
        'GBE Revenues': None,
        'GBE Expenses': None,
        'GBE Net Income': None,
        'INAC - grant contribution': None,
        'INAC - fixed contribution': None,
        'INAC - set contribution': None,
        'INAC - flexible contribution': None,
        'INAC - prior period revenue (recovery)': None,
        'INAC - current year receivable (payable)': None,
        'Health Canada': None,
        'CMHC': None,
        'Funds earned and held in trust by the federal government': None,
        'Prince Albert Grand Council': None,
        'First Nations Trust': None,
        'Northern Lights Community Development Corporation': None,
        'Government of Saskatchewan': None,
        'PAFN Business Developments LP dividends': None,
        'Northern Lights School Division': None,
        'Native claims loan agreement advances': None,
        'Land claim - future settlement (Note 26)': None,
        'Rental income': None
    }

    # Extract data using OCR
    for keyword in data:
        index = text.find(keyword)
        if index != -1:
            start_index = index + len(keyword) + 1
            end_index = text.find('\n', start_index)
            value = text[start_index:end_index].strip()
            data[keyword] = value

    # Find similar keywords using NLTK WordNet synonyms
    for keyword in data:
        if data[keyword] is None:
            synonyms = wordnet.synsets(keyword)
            similar_keywords = []
            for syn in synonyms:
                for lemma in syn.lemmas():
                    similar_keywords.append(lemma.name())
            similar_keywords = list(set(similar_keywords))
            for similar_keyword in similar_keywords:
                if similar_keyword in data:
                    data[keyword] = data[similar_keyword]
                    break

    return data


def fetch_financial_data(band_number, fiscal_year):
    base_url = "https://fnp-ppn.aadnc-aandc.gc.ca/fnp/Main/Search/DisplayBinaryData.aspx"
    query_params = {
        "BAND_NUMBER_FF": str(band_number),
        "FY": fiscal_year,
        "DOC": "Audited consolidated financial statements",
        "lang": "eng"
    }

    response = requests.get(base_url, params=query_params)
    if response.status_code == 200:
        content_type = response.headers.get('content-type')
        if content_type.startswith('application/pdf'):  # Readable PDF
            pdf_path = "financial_statements.pdf"
            with open(pdf_path, 'wb') as file:
                file.write(response.content)
            extracted_text = extract_text_from_pdf(pdf_path)
        else:  # Scanned PDF (image-based)
            pdf_image = Image.open(BytesIO(response.content))
            pdf_image.save("financial_statements.png")
            extracted_text = extract_text_from_image("financial_statements.png")

        financial_data = extract_financial_data(extracted_text)
        return financial_data

    else:
        print("Error: Unable to fetch the financial data.")
        return None


# Example usage
band_number = input("Enter Band Number: ")
fiscal_year = input("Enter Fiscal Year (yyyy-yyyy): ")

financial_data = fetch_financial_data(band_number, fiscal_year)

if financial_data is not None:
    print("Financial Data:")
    for key, value in financial_data.items():
        print(f"{key}: {value}")

Enter Band Number: 452
Enter Fiscal Year (yyyy-yyyy): 2016-2017
Financial Data:
Restricted cash - capital projects (Note 3): None
Cash and Cash equivalents: None
Cash (Note 2): 
Other revenue: 635,630 1,449,447 1,247,630
Economic Development: 64,199 145,004 172,405
Housing: 128,777 264,833 173,227
Cash and Equivalents: None
Restricted Cash: None
Accounts Receivables (AR): None
Inventory: None
Long-term Investments: None
Investment in GBEs: None
Related Party Loans: None
Investment in Trust Funds: None
Other Assets: None
Total Financial Assets: None
Bank Indebtedness: None
Accounts Payable: None
Related Part Loans: None
Deferred Revenue: 
Short-term Debt: None
Long-term Debt: None
Other Liabilities: None
Total Financial Liabilities: None
Net Financial Assets: 
Total Net Financial Assets: None
Non-Financial Assets: None
Tangible Capital Assets: 
Prepaid Expenses: None
Other non-Financial Assets: None
Total non-Financial Assets: None
Accumulated Surplus: 
Income Statement: None
Revenue: 
